In [1]:
import json
import pandas as pd

In [2]:
dataset_path = "/matx/u/simonguo/triton_sft_data/pytorch_scrape_github_inductor_data_alpaca_inst_5000_samples.json"




In [3]:
df = pd.read_json(dataset_path)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   instruction  5000 non-null   object
 1   input        5000 non-null   object
 2   output       5000 non-null   object
 3   entry_point  5000 non-null   object
 4   uuid         5000 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 195.4+ KB


In [5]:
df.head(5)

,instruction,input,output,entry_point,uuid
0,You write custom Triton kernels to replace the...,import torch\nimport torch.nn as nn\n\n\nclass...,import torch\nimport triton\nimport triton.lan...,SumAggregator,0
1,You write custom Triton kernels to replace the...,import math\nimport torch\nimport torch.utils....,import torch\nfrom torch._inductor.select_algo...,LinearEmbedding,1
2,You write custom Triton kernels to replace the...,import torch\nimport torch.nn as nn\n\n\nclass...,import torch\nimport triton\nimport triton.lan...,CustomizeLayer,2
3,You write custom Triton kernels to replace the...,import torch\nimport torch.nn as nn\n\n\nclass...,import torch\nimport triton\nimport triton.lan...,LayerNorm,3
4,You write custom Triton kernels to replace the...,import torch\nimport torch.utils.data\nimport ...,import torch\nimport triton\nimport triton.lan...,LayerNorm,4


In [6]:
first_row = df.iloc[0]
first_row






instruction    You write custom Triton kernels to replace the...
input          import torch\nimport torch.nn as nn\n\n\nclass...
output         import torch\nimport triton\nimport triton.lan...
entry_point                                        SumAggregator
uuid                                                           0
Name: 0, dtype: object

In [10]:
print(first_row["instruction"])

You write custom Triton kernels to replace the pytorch operators in the given architecture to get speedups.

You have complete freedom to choose the set of operators you want to replace. You may make the decision to replace some operators with custom Triton kernels and leave others unchanged. You may replace multiple operators with custom implementations, consider operator fusion opportunities (combining multiple operators into a single kernel, for example, combining matmul+relu), or algorithmic changes (such as online softmax). You are only limited by your imagination.

Optimize the architecture named SumAggregator with custom Triton kernels! Name your optimized output architecture SumAggregatorNew. Output the new code in codeblocks. 
Please generate real code, NOT pseudocode, make sure the code compiles and is fully functional. Just output the new model code, no other text, and NO testing code! 

            
You are given the following architecture: 




In [8]:
print(first_row["input"])

import torch
import torch.nn as nn


class SumAggregator(nn.Module):

    def __init__(self):
        super(SumAggregator, self).__init__()

    def forward(self, neighbor):
        return torch.sum(neighbor, dim=1)


def get_inputs():
    return [torch.rand([4, 4, 4, 4])]


def get_init_inputs():
    return [[], {}]



In [9]:
print(first_row["output"])

import torch
import triton
import triton.language as tl
from torch._inductor.runtime.triton_heuristics import grid
from torch._C import _cuda_getCurrentRawStream as get_raw_stream
import torch.nn as nn
assert_size_stride = torch._C._dynamo.guards.assert_size_stride
empty_strided_cuda = torch._C._dynamo.guards._empty_strided_cuda


@triton.jit
def triton_poi_fused_sum_0(in_ptr0, out_ptr0, xnumel, XBLOCK: tl.constexpr):
    xnumel = 64
    xoffset = tl.program_id(0) * XBLOCK
    xindex = xoffset + tl.arange(0, XBLOCK)[:]
    xmask = xindex < xnumel
    x0 = xindex % 16
    x1 = xindex // 16
    x2 = xindex
    tmp0 = tl.load(in_ptr0 + (x0 + 64 * x1), xmask)
    tmp1 = tl.load(in_ptr0 + (16 + x0 + 64 * x1), xmask)
    tmp3 = tl.load(in_ptr0 + (32 + x0 + 64 * x1), xmask)
    tmp5 = tl.load(in_ptr0 + (48 + x0 + 64 * x1), xmask)
    tmp2 = tmp0 + tmp1
    tmp4 = tmp2 + tmp3
    tmp6 = tmp4 + tmp5
    tl.store(out_ptr0 + x2, tmp6, xmask)


def call(args):
    arg0_1, = args
    args.clear()
 